In [ ]:
!unzip "/content/drive/MyDrive/Copy of vgg_dataset.zip"

In [2]:
import os
import csv
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Input, concatenate

# Step 1: Parse Annotations File

annotations_path = '/content/vgg_dataset/annotations.txt'

annotations = []
with open(annotations_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        parts = line.strip().split(',')
        image_path, label, x, y, width, height = parts
        annotations.append({
            'image_path': image_path,
            'label': label,
            'x': float(x),
            'y': float(y),
            'width': float(width),
            'height': float(height)
        })

In [3]:
print(annotations)

[{'image_path': 'pothole12.jpg', 'label': 'Pothole', 'x': 81.46258503401361, 'y': 133.91496598639455, 'width': 159.0, 'height': 81.99999999999999}, {'image_path': 'pothole100.jpg', 'label': 'Pothole', 'x': 133.86111111111111, 'y': 91.05555555555557, 'width': 170.0, 'height': 127.0}, {'image_path': 'pothole404.jpg', 'label': 'Pothole', 'x': 90.36111111111111, 'y': 55.055555555555564, 'width': 173.0, 'height': 109.0}, {'image_path': 'pothole319.jpg', 'label': 'Pothole', 'x': 158.67524115755626, 'y': 85.06913183279741, 'width': 164.0, 'height': 76.0}, {'image_path': 'pothole319.jpg', 'label': 'Pothole', 'x': 290.67524115755623, 'y': 66.06913183279741, 'width': 44.0, 'height': 33.99999999999999}, {'image_path': 'pothole128.jpg', 'label': 'Pothole', 'x': 138.4777158774373, 'y': 155.79944289693594, 'width': 58.999999999999986, 'height': 27.0}, {'image_path': 'pothole251.jpg', 'label': 'Pothole', 'x': 224.0772357723577, 'y': 70.44715447154472, 'width': 245.0, 'height': 97.0}, {'image_path': '

In [14]:
for annotation in annotations:
    print(annotation['label'])

Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole
Pothole


In [5]:
# Step 2: Load Images and Bounding Boxes

images_folder = '/content/vgg_dataset/images'
output_size = (299, 299)  # VGG16 input size

images = []
bounding_boxes = []
labels = []

for annotation in annotations:
    image_path = os.path.join(images_folder, annotation['label'], annotation['image_path'])
    image = Image.open(image_path).resize(output_size)
    images.append(np.array(image))

    x = annotation['x'] / image.width
    y = annotation['y'] / image.height
    width = annotation['width'] / image.width
    height = annotation['height'] / image.height
    bounding_boxes.append([x, y, width, height])

    # Assign labels (assuming binary classification: 1 for potholes, 0 for speed breakers)
    labels.append(1 if annotation['label'] == 'Pothole' else 0)

images = np.array(images)
bounding_boxes = np.array(bounding_boxes)
labels = np.array(labels)

In [6]:
print(labels)

[1 1 1 ... 0 0 0]


In [7]:
# @title
import tensorflow as tf
import tensorflow.keras

from tensorflow.keras import models, layers
from tensorflow.keras.models import Model, model_from_json, Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, SeparableConv2D, UpSampling2D, BatchNormalization, Input, GlobalAveragePooling2D

from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.utils import to_categorical

In [8]:
# @title
def entry_flow(inputs):
    x = Conv2D(32, 3, strides = 2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(64,3,padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    previous_block_activation = x
    for size in [128, 256, 728] :
        x = Activation('relu')(x)
        x = SeparableConv2D(size, 3, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(size, 3, padding='same')(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D(3, strides=2, padding='same')(x)

        residual = Conv2D(size, 1, strides=2, padding='same')(previous_block_activation)

        x = tensorflow.keras.layers.Add()([x, residual])
        previous_block_activation = x

    return x


def middle_flow(x, num_blocks=8):
    previous_block_activation = x

    for _ in range(num_blocks) :

        x = Activation('relu')(x)
        x = SeparableConv2D(728, 3, padding='same')(x)
        x = BatchNormalization()(x)

        x = Activation('relu')(x)
        x = SeparableConv2D(728, 3, padding='same')(x)
        x = BatchNormalization()(x)

        x = Activation('relu')(x)
        x = SeparableConv2D(728, 3, padding='same')(x)
        x = BatchNormalization()(x)

        x = tensorflow.keras.layers.Add()([x, previous_block_activation])
        previous_block_activation = x

    return x

def exit_flow(x) :

    previous_block_activation = x

    x = Activation('relu')(x)
    x = SeparableConv2D(728, 3, padding='same')(x)
    x = BatchNormalization()(x)

    x = Activation('relu')(x)
    x = SeparableConv2D(1024, 3, padding='same')(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D(3, strides=2, padding='same')(x)

    residual = Conv2D(1024, 1, strides=2, padding='same')(previous_block_activation)
    x = tensorflow.keras.layers.Add()([x, residual])

    x = Activation('relu')(x)
    x = SeparableConv2D(728, 3, padding='same')(x)
    x = BatchNormalization()(x)

    x = Activation('relu')(x)
    x = SeparableConv2D(1024, 3, padding='same')(x)
    x = BatchNormalization()(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)

    return x

In [9]:
from tensorflow.keras.layers import Input, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, Dropout, Conv2D
from tensorflow.keras.models import Model
import tensorflow as tf

# Define the input shape
input_shape = (299, 299, 3)  # Assuming 299x299 input size for Xception

# Define the input tensors
image_input = Input(shape=input_shape, name='image_input')
bbox_input = Input(shape=(4,), name='bounding_box_input')

# Entry Flow
x = entry_flow(image_input)

# Middle Flow
x = middle_flow(x)

# Exit Flow
x = exit_flow(x)

# Concatenate the output of exit flow with bounding box input
merged_output = tf.keras.layers.concatenate([x, bbox_input])

# Classification head
class_output = Dense(1, activation='sigmoid', name='classification')(merged_output)

# Regression head for bounding box coordinates
bbox_output = Dense(4, activation='linear', name='bounding_box')(merged_output)

# Create the model
xception_with_bbox_model = Model(inputs=[image_input, bbox_input], outputs=[class_output, bbox_output])

# Compile the model
xception_with_bbox_model.compile(optimizer='adam',
                                loss={'classification': 'binary_crossentropy', 'bounding_box': 'mean_squared_error'},
                                metrics={'classification': 'accuracy'})

# Print the model summary
xception_with_bbox_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 150, 150, 32)         896       ['image_input[0][0]']         
                                                                                                  
 batch_normalization (Batch  (None, 150, 150, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 150, 150, 32)         0         ['batch_normalization[0][0

In [10]:
# Step 4: Train the Model

from sklearn.model_selection import train_test_split

X_train, X_test, y_train_class, y_test_class, y_train_bbox, y_test_bbox = train_test_split(images, labels, bounding_boxes, test_size=0.2, random_state=42)

xception_with_bbox_model.fit([X_train, y_train_bbox], {'classification': y_train_class, 'bounding_box': y_train_bbox}, epochs=100, batch_size=32, validation_data=([X_test, y_test_bbox], {'classification': y_test_class, 'bounding_box': y_test_bbox}))

Epoch 1/100
47/47 [==============================] - 105s 1s/step - loss: 2.2214 - classification_loss: 0.6857 - bounding_box_loss: 1.5357 - classification_accuracy: 0.5074 - val_loss: 2.1034 - val_classification_loss: 0.7050 - val_bounding_box_loss: 1.3984 - val_classification_accuracy: 0.4220
Epoch 2/100
47/47 [==============================] - 44s 948ms/step - loss: 1.9267 - classification_loss: 0.6715 - bounding_box_loss: 1.2552 - classification_accuracy: 0.5539 - val_loss: 1.7930 - val_classification_loss: 0.6822 - val_bounding_box_loss: 1.1108 - val_classification_accuracy: 0.5000
Epoch 3/100
47/47 [==============================] - 45s 961ms/step - loss: 1.6994 - classification_loss: 0.6540 - bounding_box_loss: 1.0454 - classification_accuracy: 0.5836 - val_loss: 1.5806 - val_classification_loss: 0.6678 - val_bounding_box_loss: 0.9128 - val_classification_accuracy: 0.5511
Epoch 4/100
47/47 [==============================] - 45s 954ms/step - loss: 1.5321 - classification_loss: 0.

In [11]:
xception_with_bbox_model.save("xception.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
'''import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_predictions(model, X_test, y_test_bbox, y_test_class, threshold=0.5):
    predictions_class, predictions_bbox = model.predict([X_test, y_test_bbox])

    for i in range(len(X_test)):
        image = X_test[i]
        true_class = y_test_class[i]
        pred_class = predictions_class[i][0]
        pred_bbox = predictions_bbox[i]

        if pred_class > threshold:
            predicted_class = 1
            color = 'green' if true_class == 1 else 'red'

            # Scale bounding box from [0, 1] to image dimensions
            x, y, width, height = pred_bbox
            x *= image.shape[1]
            y *= image.shape[0]
            width *= image.shape[1]
            height *= image.shape[0]

            # Display bounding box only for predicted class
            plt.figure(figsize=(8, 8))
            plt.imshow(image)
            rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor=color, facecolor='none')
            plt.gca().add_patch(rect)
            plt.text(x, y - 10, f"Class: {predicted_class} (Confidence: {pred_class:.2f})", fontsize=12, color=color)
            plt.axis('off')
            plt.show()

# Call the function to visualize predictions
visualize_predictions(xception_with_bbox_model, X_test, y_test_bbox, y_test_class, threshold=0.5)'''

In [ ]:
def visualize_predictions(model, X_test, y_test_bbox, y_test_class, threshold=0.5):
    class_names = {0: 'speedbreaker', 1: 'Pothole'}  # Mapping class indices to class names

    predictions_class, predictions_bbox = model.predict([X_test, y_test_bbox])

    for i in range(len(X_test)):
        image = X_test[i]
        true_class = y_test_class[i]
        pred_class_prob = predictions_class[i][0]
        pred_bbox = predictions_bbox[i]

        if pred_class_prob > threshold:
            predicted_class_idx = 1
            color = 'green' if true_class == 1 else 'red'

            # Scale bounding box from [0, 1] to image dimensions
            x, y, width, height = pred_bbox
            x *= image.shape[1]
            y *= image.shape[0]
            width *= image.shape[1]
            height *= image.shape[0]

            # Display bounding box only for predicted class
            plt.figure(figsize=(8, 8))
            plt.imshow(image)
            rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor=color, facecolor='none')
            plt.gca().add_patch(rect)
            plt.text(x, y - 10, f"Class: {class_names[predicted_class_idx]} (Confidence: {pred_class_prob:.2f})", fontsize=12, color=color)
            plt.axis('off')
            plt.show()

# Call the function to visualize predictions
visualize_predictions(xception_with_bbox_model, X_test, y_test_bbox, y_test_class, threshold=0.5)


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

predictions_class, _ = xception_with_bbox_model.predict([X_test, y_test_bbox])
binary_predictions = (predictions_class > 0.5).astype(int)

accuracy = accuracy_score(y_test_class, binary_predictions)
precision = precision_score(y_test_class, binary_predictions)
recall = recall_score(y_test_class, binary_predictions)
f1 = f1_score(y_test_class, binary_predictions)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

12/12 [==============================] - 3s 238ms/step
Accuracy: 0.92
Precision: 0.99
Recall: 0.88
F1-score: 0.93
